# set up packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("moive analysis").config("spark.some.config.option", "some-value").getOrCreate()

# load data

In [4]:
location = "D:/py_movie_recommendation/data/"

In [5]:
movies_df = spark.read.load(location+"movies.csv", format='csv', header = True)
ratings_df = spark.read.load(location+"ratings.csv", format='csv', header = True)
links_df = spark.read.load(location+"links.csv", format='csv', header = True)
tags_df = spark.read.load(location+"tags.csv", format='csv', header = True)

In [6]:
# overview
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [7]:
ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [8]:
links_df.show(5)

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows



In [9]:
tags_df.show(5)

+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|     2|  60756|          funny|1445714994|
|     2|  60756|Highly quotable|1445714996|
|     2|  60756|   will ferrell|1445714992|
|     2|  89774|   Boxing story|1445715207|
|     2|  89774|            MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows



In [10]:
# group
tmp1 = ratings_df.groupBy("userID").count().agg({'count':'min'}).toPandas()
tmp2 = ratings_df.groupBy("movieId").count().agg({'count':'min'}).toPandas()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1.iloc[0,0]))
print('Minimum number of ratings per movie is {}'.format(tmp2.iloc[0,0]))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1


In [11]:
tmp3 = ratings_df.groupBy("movieId").count().where('count=1').count()
tmp4 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp3, tmp4))

3446 out of 9724 movies are rated by only one user


# EDA

In [12]:
# make all data into tables
movies_df.createOrReplaceTempView("movies")
ratings_df.createOrReplaceTempView("ratings")
links_df.createOrReplaceTempView("links")
tags_df.createOrReplaceTempView("tags")

## number of users

In [13]:
# here the users come from two sections, ratings and tags
# however there are still user who did not add tag or rate a movie but we can ignore those part since we cannot find them based on bd
number_user = spark.sql(
    '''
    SELECT COUNT(DISTINCT userID)
    FROM (
        SELECT DISTINCT userID
        FROM ratings

        UNION

        SELECT DISTINCT userID
        FROM tags
    )
    '''
).toPandas()
print("Total number of users we can observe is ", number_user.iloc[0, 0])

Total number of users we can observe is  610


## number of movies

In [14]:
# the movie table should have all the movie data we need
number_movie = spark.sql(
    """
    SELECT
    """
)

ParseException: 
mismatched input '<EOF>' expecting {'(', 'ADD', 'ALTER', 'ANALYZE', 'CACHE', 'CLEAR', 'COMMENT', 'COMMIT', 'CREATE', 'DELETE', 'DESC', 'DESCRIBE', 'DFS', 'DROP', 'EXPLAIN', 'EXPORT', 'FROM', 'GRANT', 'IMPORT', 'INSERT', 'LIST', 'LOAD', 'LOCK', 'MAP', 'MERGE', 'MSCK', 'REDUCE', 'REFRESH', 'REPLACE', 'RESET', 'REVOKE', 'ROLLBACK', 'SELECT', 'SET', 'SHOW', 'START', 'TABLE', 'TRUNCATE', 'UNCACHE', 'UNLOCK', 'UPDATE', 'USE', 'VALUES', 'WITH'}(line 3, pos 4)

== SQL ==


    
----^^^
